In [ ]:
from pydantic import BaseModel
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import numpy as np
from numpy.typing import NDArray
import math as m

EXAMPLE = "../example.txt"
EXAMPLE_TREE = "../example_tree.txt"
INPUT = "../input.txt"

In [ ]:
EXAMPLE_H=7
EXAMPLE_W=11
INPUT_H=103
INPUT_W=101

In [ ]:
class Coords(BaseModel):
    x: int
    y: int

In [ ]:
class Robot():
    position: Coords
    velocity: Coords

    def __init__(self, position, velocity):
        self.position = position
        self.velocity = velocity

    def quadrant(self, height, width):
        quadrant_x, quadrant_y = -1, -1
        if self.position.x < width // 2:
            quadrant_x = 0
        elif self.position.x > width // 2:
            quadrant_x = 1
        if self.position.y < height // 2:
            quadrant_y = 0
        elif self.position.y > height // 2:
            quadrant_y = 1
        return Coords(x=quadrant_x, y=quadrant_y)
    
    def move(self, seconds, height, width):
        self.position.x += (self.velocity.x * seconds)
        self.position.x = self.position.x % width
        self.position.y += (self.velocity.y * seconds) % height
        self.position.y = self.position.y % height

In [ ]:
class Swarm():
    height: int
    width: int
    robots: list[Robot]
    quadrants: dict[tuple[int, int], int]
    grid: list[list[int]]
    size: int

    def __init__(self, input_file_name, height, width):
        self.height = height
        self.width = width
        self.init_quadrants()
        self.grid = [[0 for _ in range(self.width)] for _ in range(self.height)]
        self.robots = []
        with open(input_file_name, 'r') as f:
            self.parse_input(f)

    def parse_input(self, f):
        for line in f:
            p, v = line.strip().replace("\n", "").split(" ")
            _, p = p.split('=')
            px, py = p.split(',')
            position = Coords(x=int(px), y=int(py))
            _, v = v.split('=')
            vx, vy = v.split(',')
            velocity = Coords(x=int(vx), y=int(vy))
            robot = Robot(position=position, velocity=velocity)
            self.robots.append(robot)
            self.grid[robot.position.y][robot.position.x] += 1
            quadrant = robot.quadrant(self.height, self.width)
            if (quadrant.x, quadrant.y) in self.quadrants:
                self.quadrants[(quadrant.x, quadrant.y)] += 1
        self.size = len(self.robots)

    def init_quadrants(self):
        self.quadrants = {}
        for i in [0, 1]:
            for j in [0, 1]:
                self.quadrants[(i, j)] = 0

    def move(self, seconds):
        for robot in self.robots:
            old_quadrant = robot.quadrant(self.height, self.width)
            if (old_quadrant.x, old_quadrant.y) in self.quadrants:
                self.quadrants[(old_quadrant.x, old_quadrant.y)] -= 1
            self.grid[robot.position.y][robot.position.x] -= 1
            robot.move(seconds, self.height, self.width)
            self.grid[robot.position.y][robot.position.x] += 1
            new_quadrant = robot.quadrant(self.height, self.width)
            if (new_quadrant.x, new_quadrant.y) in self.quadrants:
                self.quadrants[(new_quadrant.x, new_quadrant.y)] += 1

    def safety_factor(self):
        return m.prod(self.quadrants.values())
    
    def get_grid(self):
        return np.array(self.grid)
    
    def is_one_quadrant_bigger(self):
        if any(nb >= self.size / 2 for nb in self.quadrants.values()):
            return True
        return False

In [ ]:
def part_1(input_file_name, height, width):
    swarm = Swarm(input_file_name, height, width)
    swarm.move(100)
    result = swarm.safety_factor()
    print(result)

In [ ]:
part_1(EXAMPLE, EXAMPLE_H, EXAMPLE_W)

In [ ]:
part_1(INPUT, INPUT_H, INPUT_W)

For Part 2, best way I found is to 
- generate a gif of the robot positions
- notice that every 100 frames or so, they seem to congregate
- generate a gif with these specific frames
- notice a goddamn christmas tree in one frame

In [ ]:
class VectorizedSwarm():
    height: int
    width: int
    robots: NDArray # 3-tensor (N, 2, 2): [robot_index, position_or_velocity, x_or_y]
    quadrants: NDArray
    size: int
    grid: NDArray # 2D grid (height x width) with the count of robots at each position

    def __init__(self, input_file_name, height, width):
        self.height = height
        self.width = width
        self.robots = []
        self.quadrants = np.array([0, 0, 0, 0])
        self.grid = np.zeros((height, width), dtype=int)
        with open(input_file_name, 'r') as f:
            self.parse_input(f)
        self.update_quadrants()
        self.update_grid()

    def parse_input(self, f):
        robots = []
        for line in f:
            p, v = line.strip().replace("\n", "").split(" ")
            _, p = p.split('=')
            px, py = p.split(',')
            position = [int(px), int(py)]
            _, v = v.split('=')
            vx, vy = v.split(',')
            velocity = [int(vx), int(vy)]
            robot = [position, velocity]
            robots.append(robot)
        self.size = len(robots)
        self.robots = np.array(robots)

    def update_quadrants(self):
        self.quadrants[:] = 0
        half_width = self.width // 2
        half_height = self.height // 2
        px, py = self.robots[:, 0, 0], self.robots[:, 0, 1]
        # Compute mask of robots to count
        valid = (px != half_width) & (py != half_height)
        # Compute quadrant index of each robot
        quadrant_indices = (px[valid] > half_width) * 2 + (py[valid] > half_height)
        # Count nb of robots for each quadrant index
        counts = np.bincount(quadrant_indices, minlength=4)
        self.quadrants[:] = counts

    def update_grid(self):
        self.grid[:] = 0
        px, py = self.robots[:, 0, 0], self.robots[:, 0, 1]
        np.add.at(self.grid, (py, px), 1)

    def move(self, seconds):
        self.robots[:, 0, :] += self.robots[:, 1, :]*seconds
        self.robots[:, 0, 0] %= self.width
        self.robots[:, 0, 1] %= self.height
        self.update_quadrants()
        self.update_grid()

    def safety_factor(self):
        return m.prod(self.quadrants)
    
    def is_one_quadrant_bigger(self):
        if any(nb >= self.size / 2 for nb in self.quadrants):
            return True
        return False

In [ ]:
def generate_gif(input_file_name, height, width, frames=10):
    fig, ax = plt.subplots()

    def update(frame, swarm: Swarm):
        ax.clear()
        grid = swarm.grid
        # Move the swarm 101 times, to get the next special frame
        swarm.move(101)
        im = ax.imshow(grid, cmap='viridis', interpolation='nearest')
        ax.set_title(f"Frames: {frame}")
        return im,

    swarm = VectorizedSwarm(input_file_name, height, width)
    # Move the swarm 8 times, to get the initial special frame
    swarm.move(8)
    ani = FuncAnimation(fig, update, frames=frames, fargs=(swarm,), interval=100, repeat=False)
    ani.save("robot_dance.gif", writer=PillowWriter(fps=10))

In [ ]:
def part_2():
    generate_gif(INPUT, INPUT_H, INPUT_W, frames = 70)

In [ ]:
part_2()

The christmas tree appears after 6876 seconds.
Let's try and find that automatically, without looking at images.

In [ ]:
def part_2(input_file_name, height, width):
    swarm = VectorizedSwarm(input_file_name, height, width)
    result = 0
    while not swarm.is_one_quadrant_bigger():
        swarm.move(1)
        result += 1
    print(result)

In [ ]:
part_2(INPUT, INPUT_H, INPUT_W)